<h1>Quick start of rainfall time series generation</h1>

<p style="font-size: 20px;"> In this notebook. We will demonstrates a typical process to generate rainfall time series stochastically. </p>

<h2><b>Loading Data</b></h2>

<p>We prepared two rainfall time series data from bochum and elmdon.</p>
<ul>
    <li><b>Bochum</b>: <b>5-min</b> intensity data from <code>1931-01-01</code> to <code>1999-12-31</code>.</li>
    <li><b>Elmdon</b>: <b>1-hour</b> intensity data from <code>1980-01-01</code> to <code>1996-12-31</code>.</li>
</ul>

<p>Data was zipped under the <code>/example</code> directory.</p>

<h3>Unzip Data</h3>
<p>
Unzip data using python bult-in <code>zipfile</code> package. We will get a data foler with 2 csv files in it.<br><br>
<b>If something went wrong. Please unzip the file manually and place it inside the <code>/examples/quick_start</code> folder.
</b></p>

<pre>
pyBL/
├── examples/
    ├── <b>data/</b>
    |   ├── <b>bochum.csv</b>
    |   ├── <b>elmdon.csv</b>
    ├── quick_start.ipynb
</pre>

In [1]:
import zipfile
import os

current_path = os.getcwd()
previous_path = os.path.dirname(current_path)

with zipfile.ZipFile(previous_path + '/data.zip', 'r') as zip_ref:
    zip_ref.extractall(current_path)
    print('Data extracted successfully to ' + current_path)

Data extracted successfully to /home/chilling/git_repo/personal/pyBL/examples/quick_start


<h3>Read CSV data file</h3>
<p>
We use pandas to read the csv file. The Bochum data have a <code>datatime</code> column and a <code>Bochum</code> column.  
  
We use <code>datatime</code> column as index and parse it into <code>DatatimeIndex</code> with <code>parse_dates</code> is <code>True</code>.  
  
Loaded data was stored as a <code>pandas.Series</code> object for later useage.
</p>

In [2]:
import pandas as pd

data_name = 'bochum.csv'
data_path = current_path + '/data/' + data_name
data = pd.read_csv(data_path, index_col='datatime', parse_dates=True)['Bochum']
data

datatime
1931-01-01 00:00:00    0.0
1931-01-01 00:05:00    0.0
1931-01-01 00:10:00    0.0
1931-01-01 00:15:00    0.0
1931-01-01 00:20:00    0.0
                      ... 
1999-12-31 23:35:00    0.0
1999-12-31 23:40:00    0.0
1999-12-31 23:45:00    0.0
1999-12-31 23:50:00    0.0
1999-12-31 23:55:00    0.0
Name: Bochum, Length: 7258176, dtype: float64

<h2><b>Pre processing</b></h2>
<p>After the data was loaded as a <code>pandas.Series</code> object.<br><br>
We need to group data by connecting the data from the same month of each year to form <code>12</code> time series.<br><br>
Then, for each month, we upscale the resolution from <code>5-min</code> to <code>1-hr</code>, <code>6-hr</code>, <code>1-day</code>. To this point, <code>48</code> timeseries was created.<br><br>
Next we will calculate the parameters of each timeseries required for the BL model.<br><br>
That is,
</p>
<ul>
    <li><b>Statistical Properties</b></li>
    Classic statistics matrix for each month and each timescale.<br>
    For Bochum data, we calculate following intensity statistics.<br><br>
        <ul>
            <li><b>Mean</b></li>
                <ul>
                    Mean intensity of data.
                </ul>
            <li><b>Coefficient of Variation</b></li>
                <ul>
                    Coefficient of Variation of data. 
                </ul>
            <li><b>Correlation Coefficient</b></li>
                <ul>
                    Autocorrelation of data with 1 unit time lag (The lag time is different for different timescale).
                </ul>
            <li><b>Skewness</b></li>
                <ul>
                    Skewness of data.
                </ul>
            <li><b>Probability of Dry</b></li>
                <ul>
                    Ratio of dry (no precipitation) data.
                </ul>
        </ul>
    All statistics above is calculate for every time series of the 48 time series.<br><br> 
    <li><b>Fitting Weights</b>: </li>
    For each statistical properties. A corresponding weight is calculated based on the variance of that statistic across different years.<br>

$$
\frac{1}{Variance}
$$
</ul>

Thus, the number of weight is the same as the number of statistics.<br><br>
That is an array of size <b>[12(months), 4(scales), 5(statistics)]</b>.<br><br>

We provide a helping function <code>pybl.classic_statistics()</code> that take a <code>pandas.Series</code> and a <code>list</code> of <code>timedelta</code> as parameters.<br><br>
It returns two <code>list</code> of <code>pandas.Dataframe</code> of size 12.


In [3]:
import pybl
from datetime import timedelta

# Create timescales
timescales: list[timedelta] = [
    timedelta(minutes=5), 
    timedelta(hours=1), 
    timedelta(hours=6), 
    timedelta(hours=24)
]
intensity_time_interval = timedelta(minutes=5)

stats, weights = pybl.classic_statistics(data, timescales, intensity_time_interval)
print(f"Number of Dataframes: {len(stats)}")


Number of Dataframes: 12


Let's take a look at the statistic and weight for Janurary.

In [4]:
from IPython.display import display
display(stats[0])
display(weights[0])

,StatMetrics.MEAN,StatMetrics.CVAR,StatMetrics.AR1,StatMetrics.SKEWNESS
timescale_hr,,,,
0.083333,0.007452,5.043732,0.787497,18.068964
1.000000,0.088734,3.825266,0.593094,7.471448
6.000000,0.534309,2.733484,0.402104,5.103119
24.000000,2.149698,1.828105,0.311103,3.339886


,StatMetrics.MEAN,StatMetrics.CVAR,StatMetrics.AR1,StatMetrics.SKEWNESS
timescale_hr,,,,
0.083333,60792.443607,0.371829,83.246811,0.017291
1.000000,422.519816,0.552876,79.408475,0.182264
6.000000,11.793177,1.330723,40.418051,0.714394
24.000000,0.741367,2.915678,29.175935,1.310330


As you can see. The stability of mean intensity of January in each year makes the weight extremely high.  
And the instability of skewness in short time series makes its weight significantly lower.

We then store the result in csv file for reference.

In [5]:
# Create a directory to save the results
stats_n_weights_path = current_path + '/input_stats_weights'
if not os.path.exists(stats_n_weights_path):
    os.makedirs(stats_n_weights_path)

# Save the results
for i in range(len(stats)):
    stats[i].to_csv(stats_n_weights_path + f'/stats_{i+1}.csv')
    weights[i].to_csv(stats_n_weights_path + f'/weights_{i+1}.csv')

<h2><b>Model Fitting</b></h2>
<p>
After acquiring stats and its corresponding weights. We can start fitting.<br><br>
We first create a list to store fitted BLModel.
Then, for each month, we do the following steps:
</p>

<ul>
    <li><b>Creat a BLModel</b></li>
        <ul>
            We create a BLRPRx model. A model contain a set of initial parameters to be fit.
        </ul>
    <li><b>Fit the model</b>: </li>
        <ul>
            The <code>BLRPRx.fit()</code> function take two <code>pd.Datframe</code>, which is stats and weight.<br><br>
            We then perform a set of global or local optimization algorithm will be apply to fit the stats.<br><br>
            A simple report containing the <b>objection function</b> during fitting will be return.
        </ul>
    <li><b>Save the model</b>: </li>
        <ul>
            We then append the fitted BLRPRx model and the report to later use.<br><br>
            Including series sampling and further user specific fitting method. 
        </ul>
</ul>

In [6]:
from pybl.models import BLRPRx

BLRPRx_models: list[BLRPRx] = []
results: list[dict] = []

for i in range(len(stats)):
    # Create a model
    model = BLRPRx()

    # Fit the model to the statistics using the weights
    result = model.fit(stats[i], weights[i])
    
    # Append the model to the list of models
    BLRPRx_models.append(model)
    results.append(result)
     
    print(f"Model {i+1:02d} fitted finished, Error: {result['fun']:2.5f}, Status: {result['status']}")

Model 01 fitted finished, Error: 0.51274, Status: Maximum iteration reached
Model 02 fitted finished, Error: 0.33703, Status: Success
Model 03 fitted finished, Error: 1.16522, Status: Maximum iteration reached
Model 04 fitted finished, Error: 0.95356, Status: Maximum iteration reached
Model 05 fitted finished, Error: 0.73477, Status: Maximum iteration reached
Model 06 fitted finished, Error: 0.50834, Status: Maximum iteration reached
Model 07 fitted finished, Error: 0.10069, Status: Success
Model 08 fitted finished, Error: 0.39123, Status: Success
Model 09 fitted finished, Error: 0.43158, Status: Success
Model 10 fitted finished, Error: 0.48132, Status: Success
Model 11 fitted finished, Error: 1.53225, Status: Maximum iteration reached
Model 12 fitted finished, Error: 2.83350, Status: Maximum iteration reached


<p>
    Let's take a look at our fitting results. <br><br>
    The table below shows the weighted error between the theoretical statistics of the generation results from the BLModel and the target statistics.
<p>
<table style="width:90%; border-collapse: collapse;">
    <caption>Fitting Error</caption>
  <tr>
    <th></th>
    <th>Jan.</th>
    <th>Feb.</th>
    <th>Mar.</th>
    <th>Apr.</th>
    <th>May</th>
    <th>June</th>
    <th>July</th>
    <th>Aug.</th>
    <th>Sept.</th>
    <th>Oct.</th>
    <th>Nov.</th>
    <th>Dec.</th>
  </tr>
  <tr>
    <td>Error</td>
    <td>0.5127</td>
    <td>0.3907</td>
    <td>1.1652</td>
    <td>0.9535</td>
    <td>0.7347</td>
    <td>0.5083</td>
    <td>0.1006</td>
    <td>0.3912</td>
    <td>0.3319</td>
    <td>0.4813</td>
    <td>1.5322</td>
    <td>2.8335</td>
  </tr>
</table>
<p>
If your error is lower, great. But If your error is much higher than the table. Consider rerun the cell above.<br><br>
Once you make sure the fitted model is good enough. We can extract the BLModel parameters and save them as a csv file.<br><br>
Use <code>BLRPRx.get_params().unpack()</code> to get a numpy array of parameters.

In [7]:
import numpy as np

params = np.empty((12, 7))        # 12 models, 7 BLRPRx parameters
month_abbr = ['Jan.', 'Feb.', 'Mar.', 'Apr.', 'May', 'June', 'July', 'Aug.', 'Sep.', 'Oct.', 'Nov.', 'Dec.']
params_name  = ['lambda', 'phi', 'kappa', 'alpha', 'nu', 'sigmax_mux', 'iota']

for i in range(len(BLRPRx_models)):
    params[i] = BLRPRx_models[i].get_params().unpack()

params_df = pd.DataFrame(
    params, 
    columns=params_name, 
    index=month_abbr,
    )
params_df.index.name = 'Month'

params_df.to_csv('theta.csv')
params_df


,lambda,phi,kappa,alpha,nu,sigmax_mux,iota
Month,,,,,,,
Jan.,0.012927,0.026185,0.792320,0.794972,0.182920,0.999000,0.220731
Feb.,0.012355,0.033729,1.052730,0.964164,0.229200,0.999000,0.195573
Mar.,0.015295,0.027197,0.572284,0.951548,0.154684,0.999000,0.219543
Apr.,0.011562,0.020203,0.395655,0.691426,0.118169,0.999000,0.316853
May,0.014673,0.036977,0.350605,0.634816,0.092834,0.999000,0.542172
June,0.018159,0.042270,0.194105,0.444545,0.054380,0.999000,1.095918
July,0.016706,0.029566,0.117124,0.655371,0.089661,0.999000,1.297906
Aug.,0.011815,0.024267,0.092350,0.430575,0.089086,0.999000,1.771031
Sep.,0.015602,0.068670,0.350567,0.465955,0.089918,0.999000,1.049759


<h2><b>Sampling Time Series</b></h2>
<p>
After we found the BLRPRx model parameters that can generate time series that share the same statistical properties with the real data.<br><br>
We can then sample time series from the model using <code>BLRPRx.sample()</code>.<br><br>
The <code>BLRPRx.sample_storms()</code> method takes one <code>duration_hr</code> argument, which represents the total length of sampling in <b>hours</b>.<br><br>
The return value are a timeseries and list of storms. We'll discuss about them later.<br><br>
Here we sample <b>100k</b> hours time series for every month (BLRPRx model).
</p>

In [8]:
from pybl.timeseries import IndexedSnapshot

sample_hour = 1000000
raw_timeseries: list[IndexedSnapshot]= []
raw_storms = []

for i in range(12):
    timeseries, storms = BLRPRx_models[i].sample_storms(sample_hour)
    raw_timeseries.append(timeseries)
    raw_storms.append(storms)

<h2><b>Post Processing</b></h2>
We've already sample time series from each month's BLRPRx model.<br><br>
Let's analyze them to see if they are really close to the real data in our target statistics.<br><br>

<h3>Rescaling Time Series</h3>
The time series sampled is raw data. To perform further computation. We must first rescale it.<br><br>
Remeber the <code>timescale</code> variable we created before? We pass them to the rescale method of time series.

In [9]:
rescaled_timeseries: list[list[IndexedSnapshot]] = []
for i in range(12):
    timeseries = []
    for scale in timescales:
        timeseries.append(raw_timeseries[i].rescale(scale))
    rescaled_timeseries.append(timeseries)
    

<h3>Compute Standard Statistics</h3>
Time series in different scale were created.<br><br>
Let's compute the standard statistics to check if it is close to the real time series.<br><br>

In [22]:
sampled_stats_arr = np.empty((12, 4, 4))        # 12 months, 4 timescales, 4 statistics
sampled_stats = []

for m_idx in range(12):
    for s_idx, scale in enumerate(timescales):
        sampled_stats_arr[m_idx, s_idx, 0] = rescaled_timeseries[m_idx][s_idx].mean()
        sampled_stats_arr[m_idx, s_idx, 1] = rescaled_timeseries[m_idx][s_idx].coef_variation()
        sampled_stats_arr[m_idx, s_idx, 2] = rescaled_timeseries[m_idx][s_idx].autocorr_coef(1)
        sampled_stats_arr[m_idx, s_idx, 3] = rescaled_timeseries[m_idx][s_idx].skewness()
    sampled_stats.append(pd.DataFrame(sampled_stats_arr[m_idx], columns=stats[m_idx].columns, index=stats[m_idx].index))

<p>
We've already computed the statistics of sampled time series.<br><br>
Let's compare them with the statistics of input time series<br><br>
First let's find the best and worst fitted month. And get its theoretical statistics of sample time series.  
</p>

In [15]:
error = []
for result in results:
    error.append(result['fun'])
    
best_month_idx = np.argmin(error)
worst_month_idx = np.argmax(error)

print(f"Best fitted model: {month_abbr[best_month_idx]}, Error: {error[best_month_idx]:2.5f}")
print(f"Worst fitted model: {month_abbr[worst_month_idx]}, Error: {error[worst_month_idx]:2.5f}")

best_month_stats = results[best_month_idx]['theo_stats']
worst_month_stats = results[worst_month_idx]['theo_stats']

Best fitted model: July, Error: 0.10069
Worst fitted model: Dec., Error: 2.83350


We found the best and worst fitted month.<br><br>
Let's compare the best fitted month statistics.

In [12]:
print("Input Time Series Statistics")
display(stats[best_month_idx])
print("Theoretical Time Series Statistics")
display(best_month_stats)
print("Sampled Time Series Statistics")
display(sampled_stats[best_month_idx])
print("weights")
display(weights[best_month_idx])

Input Time Series Statistics


,StatMetrics.MEAN,StatMetrics.CVAR,StatMetrics.AR1,StatMetrics.SKEWNESS
timescale_hr,,,,
0.083333,0.008827,11.441605,0.591076,39.246999
1.000000,0.106573,6.315195,0.281881,14.653136
6.000000,0.653962,3.541951,0.209589,7.111554
24.000000,2.610442,2.076908,0.124363,3.531911


Theoretical Time Series Statistics


,StatMetrics.MEAN,StatMetrics.CVAR,StatMetrics.AR1,StatMetrics.SKEWNESS
timescale_hr,,,,
0.083333,0.008965,11.379694,0.579440,37.327097
1.000000,0.107578,6.313281,0.286659,15.252381
6.000000,0.645470,3.486270,0.201017,6.927165
24.000000,2.581880,2.054524,0.115116,3.685969


Sampled Time Series Statistics


,StatMetrics.MEAN,StatMetrics.CVAR,StatMetrics.AR1,StatMetrics.SKEWNESS
timescale_hr,,,,
0.083333,0.009150,11.252222,0.576683,38.685260
1.000000,0.109803,6.233158,0.287803,15.758294
6.000000,0.658814,3.438682,0.200025,7.094000
24.000000,2.635194,2.022155,0.114656,3.682659


weights


,StatMetrics.MEAN,StatMetrics.CVAR,StatMetrics.AR1,StatMetrics.SKEWNESS
timescale_hr,,,,
0.083333,51945.837545,0.032230,48.648425,0.005754
1.000000,361.874002,0.213772,67.256307,0.050928
6.000000,10.179309,0.897280,41.580359,0.313234
24.000000,0.648332,3.597603,26.673601,1.254076


<p>
We can see that the Theoretical BLRPRx model already fits pretty well.<br><br>
And the statistics of sampled time series is also really close to the theoretical statistics.<br><br>
We've successfully generate a time series that is similar to the input time series.<br><br>
Let's take a look at the worst fitted model.
</p>

In [16]:
print("Input Time Series Statistics")
display(stats[11])
print("Theoretical Time Series Statistics")
display(worst_month_stats)
print("Sampled Time Series Statistics")
display(sampled_stats[11])
print("weights")
display(weights[11])

Input Time Series Statistics


,StatMetrics.MEAN,StatMetrics.CVAR,StatMetrics.AR1,StatMetrics.SKEWNESS
timescale_hr,,,,
0.083333,0.007458,5.136405,0.771816,27.110470
1.000000,0.089878,3.872517,0.588337,7.390518
6.000000,0.550192,2.785453,0.399961,4.955543
24.000000,2.188294,1.908431,0.293699,3.740923


Theoretical Time Series Statistics


,StatMetrics.MEAN,StatMetrics.CVAR,StatMetrics.AR1,StatMetrics.SKEWNESS
timescale_hr,,,,
0.083333,0.007547,5.293518,0.751879,17.680986
1.000000,0.090570,3.764541,0.558652,9.119753
6.000000,0.543418,2.682903,0.443883,5.597110
24.000000,2.173671,1.872369,0.256147,3.514788


Sampled Time Series Statistics


,StatMetrics.MEAN,StatMetrics.CVAR,StatMetrics.AR1,StatMetrics.SKEWNESS
timescale_hr,,,,
0.083333,0.007446,5.348918,0.752779,17.383966
1.000000,0.089351,3.802759,0.559405,9.011932
6.000000,0.536104,2.715534,0.445947,5.532511
24.000000,2.144404,1.890824,0.249787,3.497659


weights


,StatMetrics.MEAN,StatMetrics.CVAR,StatMetrics.AR1,StatMetrics.SKEWNESS
timescale_hr,,,,
0.083333,49682.712880,0.499390,126.447550,0.018770
1.000000,345.480646,0.818751,81.107232,0.172973
6.000000,9.668645,2.096997,49.451489,0.631601
24.000000,0.612780,3.920383,34.667329,1.392406


<p>
As you can see. This model fit <code>skwness</code> at 5-min timescale pretty bad.<br><br>
However the sample time series is still really close to theoretical statistics.<br><br>
As long as you can find BLRPRx model parameters that fits the input statistics well. We can sample a similar time series.<br><br>
</p>

<h3>Save Theoretical Statistics</h3>

In [19]:
# Create a directory to save the results
path = current_path + '/theoretical_stats_weights'
if not os.path.exists(path):
    os.makedirs(path)

# Save the results
for i in range(len(stats)):
    results[i]['theo_stats'].to_csv(path + f'/stats_{i+1}.csv')


<h3>Save Sample Statistics</h3>

In [23]:
# Create a directory to save the results
path = current_path + '/sample_stats_weights'
if not os.path.exists(path):
    os.makedirs(path)

# Save the results
for i in range(len(stats)):
    sampled_stats[i].to_csv(path + f'/stats_{i+1}.csv')